In [ ]:
"""
Purpose: To pull down the earliest version of thhe synapses from 11/9

"""

In [2]:
from annotationframeworkclient import FrameworkClient
from IPython import html, display

/usr/local/lib/python3.6/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
client = FrameworkClient('minnie65_phase3_v1')

In [4]:
df_20k = client.materialize.query_table('synapses_pni_2',
                                    filter_in_dict=None,
                                    filter_out_dict=None,
                                    filter_equal_dict=None,
                                    filter_spatial=None,#not yet implemented
                                    select_columns=None,#reduces the amount of data you are pulling
                                   offset=None #this is an integer that will set the offset
                                   )

In [5]:
df = client.materialize.query_table('synapses_pni_2',
                                    #filter_in_dict=dict(func_id=[58941,16062]),
                                    filter_in_dict=dict(pre_pt_root_id=[864691131771477291,864691131881790489]), #just excludes those
                                    
                                    #filter_equal_dict=dict(func_id=58941),
                                   offset=0 #this is an integer that will set the offset
                                   )
df

,id,valid,pre_pt_position,pre_pt_supervoxel_id,pre_pt_root_id,ctr_pt_position,post_pt_position,post_pt_supervoxel_id,post_pt_root_id,size


# Getting the synapses for the neurons we have 

In [6]:
from os import sys
sys.path.append("/meshAfterParty")

In [7]:
"""
All the possible fields:

id
pre_pt_position
pre_pt_supervoxel_id
pre_pt_root_id
post_pt_position
post_pt_supervoxel_id
post_pt_root_id
size


"""

'\nAll the possible fields:\n\nid\npre_pt_position\npre_pt_supervoxel_id\npre_pt_root_id\npost_pt_position\npost_pt_supervoxel_id\npost_pt_root_id\nsize\n\n\n'

In [8]:
columns_we_want = [
    "id",
"pre_pt_position",
"pre_pt_supervoxel_id",
"pre_pt_root_id",
"post_pt_position",
"post_pt_supervoxel_id",
"post_pt_root_id",
"size"
]

columns_we_want

['id',
 'pre_pt_position',
 'pre_pt_supervoxel_id',
 'pre_pt_root_id',
 'post_pt_position',
 'post_pt_supervoxel_id',
 'post_pt_root_id',
 'size']

In [9]:
import datajoint_utils as du

In [10]:
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-25 17:55:57,078 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-25 17:55:57,079 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-25 17:55:57,089 - connection - Connected celiib@at-database.ad.bcm.edu:3306


Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-25 17:55:57,365 - settings - Setting enable_python_native_blobs to True


In [11]:
seg_ids_to_check = (minnie.SegToDecimateFromNuclei()).fetch("segment_id")
seg_ids_to_check_total

NameError: name 'seg_ids_to_check_total' is not defined

In [ ]:
import numpy as np
soma_segment_ids = np.unique((minnie.BaylorSegmentCentroid & "multiplicity>0").fetch("segment_id"))

In [ ]:
len(soma_segment_ids)/step_size

In [ ]:
df = client.materialize.query_table('synapses_pni_2',
                                            #filter_in_dict=dict(func_id=[58941,16062]),
                                            filter_in_dict=curr_filter_dict, #just excludes those

                                            #filter_equal_dict=dict(func_id=58941),
                                           offset=0 ,#this is an integer that will set the offset
                                            #select_columns=columns_we_want
                                           )

In [ ]:
i

In [ ]:
"""
Pseudocode: 

in batches of 100 segment ids

for both presyn and postsyn:
1) Get the rows for that attirbute and those segments and turn into dictionaries and add to list
2a) If the result is less than 200k don't need to repeat again 
2b) If they are greater, then repeat 1 and 2 (and add offset) until less than 200k
3) Write the entries to datajoint

"""

import pandas as pd

def fix_dataframe_and_return_dict(df):
    if len(df) == 0:
        return []
    df[["centroid_x","centroid_y","centroid_z"]] = pd.DataFrame(df["ctr_pt_position"].tolist(),index=df.index)
    df_renamed = df.rename(columns=dict(pre_pt_root_id="presyn",post_pt_root_id="postsyn",id="synapse_id"))
    B = df_renamed.drop(["valid","pre_pt_position","ctr_pt_position","post_pt_position"],axis=1)
    B = B.astype({'size': "int32"})
    #print(B.dtypes)
    return B.to_dict("records")



verbose = False

step_size = 10
max_return_size = 200000
total_ids = soma_segment_ids
n_iterations = int(np.ceil(len(total_ids)/step_size))


for i in tqdm(1043,range(n_iterations)):
    if i == (n_iterations - 1):
        curr_seg_ids =  total_ids[i*step_size:]
    else:
        curr_seg_ids = total_ids[i*step_size:(i+1)*step_size]
        
    total_dicts = []
    for filter_type in ["pre_pt_root_id","post_pt_root_id"]:
        curr_filter_dict = {filter_type:curr_seg_ids}
        curr_offset = 0
        #print(f"curr_filter_dict = {curr_filter_dict}")
        while True:
            df = client.materialize.query_table('synapses_pni_2',
                                                #filter_in_dict=dict(func_id=[58941,16062]),
                                                filter_in_dict=curr_filter_dict, #just excludes those

                                                #filter_equal_dict=dict(func_id=58941),
                                               offset=curr_offset ,#this is an integer that will set the offset
                                                #select_columns=columns_we_want
                                               )
            
            df_dicts = fix_dataframe_and_return_dict(df)
            total_dicts += df_dicts
            if len(df_dicts) < max_return_size:
                break
            else:
                curr_offset += max_return_size
    
    #write the data to the datajoint table
    if len(total_dicts) > 0:
        if verbose:
            print(f"Inserting {len(total_dicts)} dictionaries ")
        SynapseTest.insert(total_dicts,skip_duplicates=True)
    else:
        if verbose:
            print("Skipping dictionary insertion because it was empty")
            
    
    #raise Exception("")
            

# Datajoint table to dump the results into 

In [ ]:
m65.Synapse.describe()

In [ ]:
{'synapse_id': 1133000,
  'pre_pt_supervoxel_id': 74535343491123341,
  'presyn': 864691134742108478,
  'post_pt_supervoxel_id': 74535343491123341,
  'postsyn': 864691134742108478,
  'size': 3316.0,
  'centroid_x': 70902,
  'centroid_y': 171240,
  'centroid_z': 14887},

In [ ]:
@schema
class SynapseTest(dj.Manual):
    definition="""
    synapse_id: bigint unsigned              # synapse index within the segmentation
    ---
    pre_pt_supervoxel_id: bigint unsigned
    presyn: bigint unsigned
    post_pt_supervoxel_id: bigint unsigned
    postsyn: bigint unsigned
    
    centroid_x: bigint unsigned
    centroid_y: bigint unsigned
    centroid_z: bigint unsigned
    
    size                 : int unsigned                 # (EM voxels)
    
    """